# Soil erodibility

<img src="images/soil_erodibility.png" style="width: 400px; float:right"><h>The soil erodibility measures the susceptibility of soil particles to detachment and transport by rainfall and runoff. It depends on soil properties such as texture, structure, organic matter content, and permeability. It is a parameter of the Universal Soil Loss Equation (USLE) and it is referred as the $K$-factor.

It can be estimated by the USDA erodibility factor equation. It is expressed in terms of the three primary components—texture $(K_1)$, structure $(K_s)$, and permeability $(K_p)$—can be written as:

$
K = K_1 + K_s + K_p
$

Where:  
- $K_1$ is the contribution of soil texture.  
- $K_s$ is the contribution of soil structure.  
- $K_p$ is the contribution of soil permeability.  

Here is the detailed expression for each component in the soil erodibility equation:

1. **Texture Component**:
   $
   K_1 = 2.1 \cdot 10^{-4} \cdot M^{1.14}(12 - \text{OM}) / 100
   $
   
   where:  
   
   - $\text{OM}$: organic matter content (%)
   - $\text{M}$: particle size parameter 
   
   $
   M = \text{silt} \cdot (100 - \text{clay})
   $
   
   - $\text{silt}$ = silt content (%)
   - $\text{clay}$ = clay content (%) 
   

2. **Structure Component**:
   
   $
   K_s = 3.25(\text{S} - 2)/100
   $
   
   where: 
   
   - $\text{S}$ = soil structure class:  


3. **Permeability Component**:
   
   $
   K_p = 2.5(\text{P} - 3)/100
   $
   
   where:  
   
   - $\text{P}$ = soil permeability class:  
   

#### **Soil Structure Classes**
The nomograph and its equation were fit to unit-plot data where structure, permeability, and splash/runoff interact. So the empirical fit ended up giving a small positive boost to $K$ for blocky, platy soils, and a small discount for very‐fine granular ones. It’s not a pure measure of “aggregate stability,” but rather an aggregate of all those field observations. In practice, texture ($M$) and organic matter swamp that structure correction, so clayey, blocky soils (low $M$, often high $OM$) still have lower 
$K$ overall than sandy, granular soils.

| **Class** | **Description**           | **Examples**                                  |
|-----------|---------------------------|-----------------------------------------------|
| 1         | Very fine granular        | Sand, Loamy sand                              |
| 2         | Fine granular             | Sandy loam, Loam, Silt, Silty loam            |
| 3         | Medium or coarse granular | Loam, Silt, Silty loam                        |
| 4         | Blocky, platy, or massive | Clay loam, Silty clay loam, Sandy clay, Clay  |

---

#### **Permeability Classes**
Soil permeability reflects the soil's ability to transmit water, affecting its susceptibility to runoff. These classes range from 1 (very slow) to 6 (very rapid):

| **Class** | **Description**    |**Texture**                | **Saturated hydraulic conductivity** |
|-----------|--------------------|---------------------------|--------------------------------------|
| 1         | Fast and very fast |Sand                       | > 61 mm/h                            |
| 2         | Moderate fast      |Loamy sand, sandy loam     | 20.3 - 61 mm/h                       |
| 3         | Moderate           |Loam, silty loam           | 5.1 - 20.3 mm/h                      |
| 4         | Moderately slow    |Sandy clay loam, clay loam | 2 - 5.1 mm/h                         |
| 5         | Slow               |Silty clay loam, sand clay | 1 - 2 mm/h                           |
| 6         | Very slow          |Silty clay, clay           | < 1 mm/h                             |

### Import necessary libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interactive, Dropdown
import matplotlib.pyplot as plt
# Import the necessary iMPACTools (you can find these tools in the Python files stored in the */iMPACtools* folder)
os.chdir('..') # change the current working directory to the parent directory
from iMPACTools.soil_texture import soil_texture_calculator, permeability_code_calculator, structure_code_calculator

### Define the Python function to compute the soil erodibility factor using the USDA equation

In [ ]:
# Function to compute the soil erodibility factor
def soil_erodibility(silt, clay, organic_matter, structure, permeability):
    """
    Calculate the soil erodibility factor (K) based on the USDA equation.

    Parameters:
    silt (float): Silt percentage in the soil.
    clay (float): Clay percentage in the soil.
    organic_matter (float): Organic matter content percentage.
    structure (int): Soil structure class (1 to 4).
    permeability (int): Soil permeability class (1 to 6).

    Returns:
    float: Soil erodibility factor (K).
    """
    # Calculate the M factor
    if silt + clay > 100:
        clay = 100 - silt
    M = (silt) * (100 - clay)
    
    # K1: Soil Texture Factor
    K1 = (2.1e-4 * (12 - organic_matter) * M ** 1.14)/ 100 * 0.1317  # Conversion to SI units
    
    # Ks: Soil Structure Factor
    Ks = 3.25 * (structure - 2)/ 100 * 0.1317  # Conversion to SI units
    
    # Kp: Soil Permeability Factor
    Kp = 2.5 * (permeability - 3)/ 100 * 0.1317  # Conversion to SI units
    
    # Total K factor
    K = (K1 + Ks + Kp)
    
    return K1, Ks, Kp, K  # Return components and total K

### Let's play with our model

In [ ]:
# Function to display the interactive plot
def plot_interactive(silt, clay, organic_matter, structure, permeability):
    
    K1, Ks, Kp, K = soil_erodibility(silt, clay, organic_matter, structure, permeability)
    
    soil_texture = soil_texture_calculator(silt,clay)
    
    # Create a figure with 4 bars (K1, Ks, Kp, Total K)
    fig, ax = plt.subplots(figsize=(7, 4))
    bar_width = 0.4  # Set width for bars
    index = [0, 0.5, 1, 2]  # Positions for the bars on the x-axis
    
    ax.bar(index[0], K1, bar_width, color="skyblue", label="K1 (Texture)")
    ax.bar(index[1], Ks, bar_width, color="lightgreen", label="Ks (Structure)")
    ax.bar(index[2], Kp, bar_width, color="lightcoral", label="Kp (Permeability)")
    ax.bar(index[3], K, bar_width*2, color="lightblue", label="Total K")
    
    ax.set_xticks(index)
    ax.set_xticklabels(["K1", "Ks", "Kp", "Total K"])
    ax.set_xlim(-0.5, 2.5)
    ax.set_ylim(-0.01, 0.1)  # Set y-axis limit to avoid bars going too high
    ax.set_ylabel('(t ha h) / (ha MJ mm)')
    
    ax.set_title(f"Soil Erodibility Factor (K): {K:.3f} - Soil texture: {soil_texture}",fontweight='bold')
    ax.legend(loc="upper left")
    plt.show()

# Mapping of Structure and Permeability Class Numbers to Descriptions
structure_classes = {
    1: "Very fine granular",
    2: "Fine granular",
    3: "Medium or coarse granular",
    4: "Blocky, platy, or massive structures"
}

permeability_classes = {
    1: "Very fast",
    2: "Moderate fast",
    3: "Moderate",
    4: "Moderate slow",
    5: "Slow",
    6: "Very slow"
}

# Interactive sliders for input values
silt_slider = widgets.FloatSlider(value=40, min=0, max=100, step=1, description="Silt (%)")
clay_slider = widgets.FloatSlider(value=20, min=0, max=100, step=1, description="Clay (%)")
organic_matter_slider = widgets.FloatSlider(value=2, min=0, max=10, step=0.1, description="Organic (%)")

# Dropdowns for structure and permeability with class names
structure_dropdown = widgets.Dropdown(
    options=[(desc, num) for num, desc in structure_classes.items()],
    value=3,
    description="Structure"
)

permeability_dropdown = widgets.Dropdown(
    options=[(desc, num) for num, desc in permeability_classes.items()],
    value=4,
    description="Permeability"
)

interactive(plot_interactive,silt = silt_slider, clay = clay_slider , organic_matter = organic_matter_slider, 
            structure = structure_dropdown,permeability = permeability_dropdown)


# Compute the soil erodibility for several sample points
This script reads soil texture data from an Excel file, computes the USDA-based RUSLE  soil erodibility factor (K) for each sample point using the provided erodibility functions, and writes the results to a new Excel file.
## Choose the case study

In [ ]:
# Get list of case studies (folders in the Case_studies directory)
case_study = Dropdown(options=os.listdir('Case_studies'),description='Case Study:')
display(case_study)  

In [ ]:
# --- Read the Excel file ---
# Expects input columns: 'ID', 'X', 'Y', 'silt', 'clay', [optional 'organic_matter']
df = pd.read_excel(f'Case_studies/{case_study.value}/soil/soil_texture.xls')

# --- Initialize the output DataFrame for K-factor results ---
K_samples = pd.DataFrame(columns=['ID','X', 'Y', 'Z'])
K_samples['ID'] = df['ID']
K_samples['X'] = df['X']
K_samples['Y'] = df['Y']
K_samples = K_samples.set_index('ID')

# --- Loop over each sample to calculate erodibility K-factor ---
for i in range(len(df)):
    
    # Extract soil texture percentages
    silt = df['silt'][i]
    clay = df['clay'][i]
    
    # Use a default organic matter content
    organic_matter = 1
    
    # Calculate structure and permeability codes from the texture
    structure = structure_code_calculator(silt, clay)
    permeability = permeability_code_calculator(silt, clay)
    
    # Compute erodibility factors: K1 (texture), Ks (structure), Kp (permeability), and total K
    # Only the total K is stored in the output DataFrame
    _,_,_, total_K = soil_erodibility(silt, clay, organic_matter, structure, permeability)
    K_samples['Z'][i] = total_K

## Save the computed K-factor results to an Excel file

##### Save the estimated erodibility

In [ ]:
# --- Save the computed K-factor results to an Excel file ---
K_samples.to_excel(f'Case_studies/{case_study.value}/soil/K_factor.xlsx')